In [1]:
import numpy as np
import pandas as pd

from scipy.stats import ranksums,pearsonr

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [2]:
PREDICTOR = 'weekly_pct'

In [3]:
df = pd.read_csv('RF_ready.csv')

split = np.floor(0.7*df.shape[0]).astype(int)

df.sort_values(by='sunday_beginning',inplace=True)

x = df
x['L3'] = (x['one']+x['two']+x['three'])/3.0
x['L5'] = (x['one']+x['two']+x['three']+x['four']+x['five'])/5.0

y = df[[PREDICTOR]]

x_train = x.iloc[:split,:]
x_test = x.iloc[split:,:]
y_train = y.iloc[:split,:]
y_test = y.iloc[split:,:]

x_train1 = x_train[['six','five','four','three','two','one','L3','L5','conm_code']]
x_test1 = x_test[['six','five','four','three','two','one','L3','L5','conm_code']]

y_train_bin = (y_train>0).astype(int)
y_test_bin = (y_test>0).astype(int)

IOError: File RF_ready.csv does not exist

In [ ]:
model = RandomForestClassifier(n_estimators=10000)

model.fit(x_train1,y_train_bin)

In [ ]:
preds = model.predict_proba(x_test1)
preds = pd.DataFrame(preds)
y_test.reset_index(drop=True,inplace=True)
preds.head()

In [ ]:
y_test['preds'] = preds[0]
y_test['binary_ret'] = (y_test[PREDICTOR]>0).astype(int)

In [ ]:
y_test.sort_values(by='preds')

In [ ]:
pearsonr(y_test[PREDICTOR],y_test['preds'])

In [ ]:
x_test.reset_index(inplace=True)
x_test['preds'] = y_test['preds']

x_test.to_csv('testset_reg.csv',index=False)

In [ ]:
preds = model.predict_proba(x_train1)
preds = pd.DataFrame(preds)
y_train.reset_index(drop=True,inplace=True)
y_train['preds'] = preds[0]
y_train['binary_ret'] = (y_train[PREDICTOR]>0).astype(int)

x_train.reset_index(inplace=True)
x_train['preds'] = y_train['preds']

x_train.to_csv('trainset_reg.csv',index=False)

In [ ]:
print np.mean(y_test[PREDICTOR])
print "and the pct positive:"
print np.mean(y_test['binary_ret'])

In [ ]:
print np.count_nonzero(y_test[y_test['preds']<0.5][PREDICTOR])
print np.mean(y_test[y_test['preds']<0.5][PREDICTOR])
print np.mean(y_test[y_test['preds']<0.5]['binary_ret'])

In [ ]:
a = np.linspace(0.50,0.7,5)

for pred in a:
    print "predicted return of "+str(pred)
    print 'count is: '+str(np.count_nonzero(y_test[y_test['preds']>pred][PREDICTOR]))
    print np.mean(y_test[y_test['preds']>pred][PREDICTOR])
    print np.mean(y_test[y_test['preds']>pred]['binary_ret'])